# Training data

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

In [ ]:
files = list(Path("/home/simonpf/data_3/ccic/training_data/").glob("cloudsat_match*.nc"))
data = xr.load_dataset(np.random.choice(files))

The retrieval output is stored in a sparse format. The variables ``profile_row_inds`` and ``profiles_col_inds`` specify the indices of the rows and columns of the 2D input image that the profiles correspond to.

In [ ]:
row_inds = data.profile_row_inds
col_inds = data.profile_column_inds

In [ ]:
iwp = data.iwp.data
iwc = data.iwc.data
cloud_class = data.cloud_class.data
tbs = data.ir_win.data[row_inds, col_inds]
cs_lats = data.latitude_cloudsat.data
cs_lons = data.longitude_cloudsat.data

In [ ]:
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LogNorm, Normalize

f = plt.figure(figsize=(8, 8))
gs = GridSpec(3, 2, width_ratios=(1.0, 0.08))

axs = np.array([f.add_subplot(gs[i, 0]) for i in range(3)])

# IWP and Tbs
x = cs_lats
ax = axs[0]
ax.plot(cs_lats, tbs, c="C1")
ax.set_ylabel("Brithness temp. [K]", color="C1")
ax = ax.twinx()
ax.plot(cs_lats, iwp, c="C0")
ax.set_yscale("log")
ax.set_ylabel("IWP [g / m^2]", color="C0")

# IWC
ax = axs[1]
iwc_norm = LogNorm(1e-3, 1e0)
y = data.levels.data
m = ax.pcolormesh(x, y, iwc.T, norm=iwc_norm)

cax = f.add_subplot(gs[1, -1])
plt.colorbar(m, cax=cax, label="IWC [g/m^3]")


# Cloud mask
ax = axs[2]
y = data.levels.data
m = ax.pcolormesh(x, y, cloud_class.T, cmap="Set2", norm=Normalize(-0.5, 7.5))

cax = f.add_subplot(gs[2, -1])
plt.colorbar(m, cax=cax, label="Cloud class")